# Model Pruning
Use this notebook to load a pre-trained Mask R-CNN model and then prune the convolutional layers to reduce the size. Save the pruned weights for later inference using the testing notebook.

Import the required libraries for loading the Mask R-CNN model and pruning it.

---



In [ ]:
import os 
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd 
import glob
from skimage.morphology import label
from skimage.io import imread
from skimage.io import imshow
import time
import tensorflow as tf
tf.get_logger().setLevel("ERROR")
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [ ]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Due to memory limitations, limit the memory that can be used on the GPU

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 4.5GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4500)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)


In [ ]:
ROOT_DIR = os.path.abspath("") # Setthe root directory of the project, where the notebook is running
DATA_DIR = os.path.join(ROOT_DIR, "datasets/airbus_data") # Set the location of the data
SHIP_CLASS_NAME = 'ship' # Declare the class name to be detected

# Set locations for training and testing data
TRAIN_DATA_DIR = os.path.join(DATA_DIR, "train_v2")
TEST_DATA_DIR = os.path.join(DATA_DIR, "test_v2")
TRAIN_SHIP_SEGMENTATIONS_PATH = os.path.join(DATA_DIR, 'train_ship_segmentations_v2.csv')

Import the Mask R-CNN framework obtained from https://github.com/akTwelve/Mask_RCNN 

In [ ]:
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

Config used for training on the dataset is required to initialise the model before pruning.

In [ ]:
class ShipDetectionConfig(Config):
    NAME = 'SHIPS'
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    BACKBONE = 'resnet50'

    NUM_CLASSES = 1+1
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 50
    SAVE_BEST_ONLY = True
    DETECTION_MIN_CONFIDENCE = 0.95
    DETECTION_NMS_THRESHOLD = 0.05
    TRAIN_ROIS_PER_IMAGE = 24
    LEARNING_RATE = 0.001

config = ShipDetectionConfig()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.95
DETECTION_NMS_THRESHOLD        0.05
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE            

Initialise the model

In [ ]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=ROOT_DIR)
model_path = os.path.join(ROOT_DIR, 'ships20201030T1506/mask_rcnn_ships_0123.h5')
model.load_weights(model_path, by_name=True)
model = model.keras_model

Display a summary of the model's layers and parameters

In [ ]:
model.summary()

Model: "mask_rcnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 518, 518, 3)  0           input_image[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 256, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________

In [ ]:
import keras
opt = keras.optimizers.SGD(lr = config.LEARNING_RATE, momentum = config.LEARNING_MOMENTUM, clipnorm = config.GRADIENT_CLIP_NORM)

Apply the pruning to specific layers during the cloning of the model and output a summary of the modified layers and their parameters.

In [ ]:
import tensorflow_model_optimization as tfmot

def apply_pruning(layer):
    
    if isinstance(layer, tf.keras.layers.Conv2D):
        return tfmot.sparsity.keras.prune_low_magnitude(layer)
    return layer

model_for_pruning = tf.keras.models.clone_model(
    model,
    clone_function=apply_pruning,
    input_tensors=model.inputs
)
model_for_pruning.summary()
base_model = model_for_pruning

Model: "mask_rcnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 518, 518, 3)  0           input_image[0][0]                
__________________________________________________________________________________________________
prune_low_magnitude_conv1 (Prun (None, 256, 256, 64) 18882       zero_padding2d[1][0]             
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 64) 0           prune_low_magnitude_conv1[0][0]  
__________________________________________________________________________________________

Re-compile the model to be trained again and pruned during training. Then train the model.

In [ ]:
import tempfile
from mrcnn.model import DataGenerator
# Define the model.
base_model = model_for_pruning

loss_names = ["rpn_class_loss", "rpn_bbox_loss", "mrcnn_class_loss", "mrcnn_bbox_loss", "mrcnn_mask_loss"]
for name in loss_names:
    layer = base_model.get_layer(name)
    if layer.output in base_model.losses:
        continue
    loss = (tf.reduce_mean(input_tensor=layer.output, keepdims=True)
           *config.LOSS_WEIGHTS.get(name, 1.))
    base_model.add_loss(loss)    

log_dir = tempfile.mkdtemp()
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    # Log sparsity and other metrics in Tensorboard.
    tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir)
]

base_model.compile(optimizer = opt, loss=[None] * len(base_model.outputs))

train_generator = DataGenerator(dataset_train, config, shuffle=True, augmentation=None)
test_generator = DataGenerator(dataset_val, config, shuffle=True)
base_model.fit(train_generator, 
               epochs=7,
               steps_per_epoch = config.STEPS_PER_EPOCH,
               callbacks=callbacks,
               validation_data = test_generator,
               validation_steps = config.VALIDATION_STEPS)
               

Epoch 1/5


/home/chris/anaconda3/envs/mrcnn2/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


100/100 [==============================] - 158s 2s/step - loss: 2.5039 - val_loss: 2.0878
Epoch 2/5
100/100 [==============================] - 162s 2s/step - loss: 2.1949 - val_loss: 2.3917
Epoch 3/5
100/100 [==============================] - 160s 2s/step - loss: 2.3093 - val_loss: 2.2739
Epoch 4/5
100/100 [==============================] - 167s 2s/step - loss: 2.4268 - val_loss: 2.9691
Epoch 5/5
100/100 [==============================] - 171s 2s/step - loss: 2.5299 - val_loss: 2.2422


Export the pruned and unpruned weights for comparison.

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(base_model)

_, pruned_keras_file = tempfile.mkstemp('.h5')
model_for_export.save_weights('pruned_weights.h5') # Save weights from the pruned model.
base_model.save_weights('weights.h5') #Save weights from the unpruned model.


In [ ]:
def get_gzipped_model_size(file):
    import os
    import zipfile
    
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)
        
    return os.path.getsize(zipped_file)

In [ ]:
print("Size of gzipped pruned model without stripping: %.2f bytes" % (get_gzipped_model_size('weights.h5')))
print("Size of gzipped pruned model with stripping: %.2f bytes" % (get_gzipped_model_size('pruned_weights.h5')))

Size of gzipped pruned model withput stripping: 195040583.00 bytes
Size of gzipped pruned model with stripping: 127424173.00 bytes
